In [2]:
!pip install pulp

In [4]:
import pulp as op
import numpy as np

In [5]:
p = np.array([1,2,3]) #precios de las variedades.
#B = np.array([[0,0,0],[0,0,0]]) #matriz binaria de camas por variedad sembrada.
w = np.array([150,250,350,20,5,12,22,1000]) #capacidad de siembra de los bancos.
v = np.array([100,350,1]) # cantidades de variedades disponibles para sembrar.
n = v.shape[0] #Cantidad de variedades.
m = w.shape[0] #Cantidad de camas.
L=np.array([[0,0,0]]) #Restricciones de NO compatibilidad por variedad.

In [15]:
conca = lambda t,e : str(t)+str(e)
bedsvars_items = list([conca(xt,xe) for xt,xe in zip(np.repeat(['BancoVar'],m*n),np.tile(np.arange(1,m),n))])
bedsvars_items

['BancoVar1',
 'BancoVar2',
 'BancoVar3',
 'BancoVar4',
 'BancoVar5',
 'BancoVar6',
 'BancoVar7',
 'BancoVar1',
 'BancoVar2',
 'BancoVar3',
 'BancoVar4',
 'BancoVar5',
 'BancoVar6',
 'BancoVar7',
 'BancoVar1',
 'BancoVar2',
 'BancoVar3',
 'BancoVar4',
 'BancoVar5',
 'BancoVar6',
 'BancoVar7']

In [ ]:
prop = op.LpProblem("Siembra Esquejes",op.LpMinimize)
beds=op.LpVariable("BedsVar",bedsvars_items,lowBound=0,upBound=1,cat='Integer')